## Alberi di regressione
Algoritmi di regressione molto professionalizzanti. Il sito web Kagool, che propone sfide in ML, viene dimostrato che richieda l'uso nel 90% dei casi di alberi di regressione.

### Caso pratico
Abbiamo un dataset contenente persone di cui conosciamo
- età
- genere
- occupazione
e altre, che sono dette *features* o variabili $X$ di input.
La variabile *target* $y$ da predire è quanto una persona apprezza i videogame.
Vogliamo creare una struttura ad albero per modellare il fenomeno, dove ogni nodo intermedio è un predicato relativo a una variabile in input
<img src="imgs/alberi.png" alt="alberi" width=500>
Questi **predicati** dividono le istanze in due sezioni: quelli che danno come risposta "sì" al predicato e quelli che danno come risposta "no". Ovviamente ogni nodo avrà dei nodi sottofigli con i rispettivi sottoalberi.
Ogni nodo foglia conterrà la risposta al quesito di partenza. Cioè questi nodi foglia conterranno tutte le istanze classificate sulla base degli $n$ quesiti o **predicati** e su ogni foglia andrà calcolata la regressione per stimare l'opportuno $y$ corrispondente. Essendo di regressione, l'obbiettivo è quello di minimizzare l'errore in questa predizione, questa volta in *ogni* foglia. Cioè in ogni foglia ci sarà la regressione fatta su tutte le istanze che sono finite in quella foglia, tentando di raggrupparle in modo che il risultato di tale regressione sia il più simile fra istanze della stessa foglia possibile. Ad esempio nell'immagine sopra farò regressione selettivamente su 
- Persone di età maggiore di 15 anni, che produrrà intuitivamente un valore -1 poiché poco interessati.
- Persone di età inferiore a 15 anni e femmine, score pari a +0.1.
- Persone di età inferiore a 15 anni e maschi, otterrà score più alto pari a +2 (quelli che 
gradiscono maggiormente i videogame).
Ancora non sappiamo nulla su come sono scelti i predicati e in che ordine o su quando si è arrivati alle foglie o bisogna ulteriormente proseguire.

### Come funziona 
Proviamo ora a comprendere il meccanismo di funzionamento di questi algoritmi, partendo da una versione più semplice (a una sola feature) per poi estenderla a due feature e infine al caso generale
#### Singola feature
Consideriamo ora un dataset composto da una sola variabile $X$ che crei un albero binario (cioè che quando si splitta ha sia nodo figlio destro che sinistro). Osservando la seguente immagine
<img src="imgs/alberireg.png" alt="albero binario" width=400>
si vede lo spazio di dati su un grafico sul quale l'asse delle $x$ è per i dati che abbiamo, cioè $X$ e $y$ per la variabile da predire. Quello che si fa è iniziare con una regressione a valore costante, dunque graficamente una retta ortogonale rispetto all'asse y (in grigio) per cui i dati vengono divisi con un certo errore (calcolabile con i minimi quadrati: cioè quanto le nostre $y$ predette si allontanano da quelle fornite come dati). Questa regressione iniziale costituisce la radice dell'albero, a questo punto l'algoritmo prosegue scegliendo un'istanza $s_1 \in X$ come elemento, potremmo dire di partizionamento del dataset. Cioè su di esso (la retta rossa) vengono suddivisi i dati e si esegue una regressione di un certo tipo a seconda di dove si trova un determinato dato rispetto a tale retta. Cioè 
- per ogni $x < s_1$ si esegue una regressione con l'elemento $y_1$.
- per ogni $x \geq s_1$ la regressione è fatta su $y_2$.
In poche parole è come se i due insiemi di dati fossero differenti, e dunque venissero due modelli completamente differenti. È evidente che in questo modo *l'aderenza del modello ai dati sarà per forza maggiore* rispetto a una regressione unica per tutto il dataset, poiché ciascuna regressione è fatta su un insieme specifico di dati.

#### Come scegliere la $x$ opportuna
In questo modello abbiamo un dataset composto da una sola feature $x$ quindi non dobbiamo preoccuparci, per il momento, di come scegliere la feature (o l'asse) su cui splittare i dati, tuttavia bisogna capire come scegliere il valore da dare a quella feature per splittare, in altre parole bisogna capire come scegliere $s_1$. Questo valore viene scelto in maniera tale da rendere minimo l'errore commesso dalla retta di regressione sui dati di valore inferiore a $s_1$ e quella commessa dalla retta di regressione per i valori maggiori uguali. In questo si cela anche il criterio d'arresto, infatti se l'errore commesso dalla retta in una delle due regioni create a partire dal primo split è troppo grande rispetto a quanto stabilito, si genera un'ulteriore split su un altro valore. Cioè si continua a dividere la regione ricorsivamente con lo stesso criterio appena visto, finché l'errore di regressione commesso su ogni regione (che concettualmente rappresenta una foglia dell'albero) è minore di un certo valore.
Ad esempio proseguendo con il dataset di prima, lo spazio di sinistra (cioè dei dati $x < s_1$) viene ulteriormente suddiviso secondo il predicato "$x < s_2$" il quale darà origine a due spazi (figli del nodo $x < s_1$)
1. dei valori inferiori a $s_2$ ($x < s_2$).
2. valori compresi fra $s_2$ e $s_1$ ($s_2 \leq x < s_1$).

in modo che le due regressioni cosi prodotte abbiano errore ancora inferiore. La cosa funziona poiché le due rette di regressione passano più vicine ai sottoinsiemi di dati della propria regione.
<img src="imgs/splitting.png" alt="Ulteriore suddivisione della regione di sinistra" width=400>
Analogamente nello spazio di destra rispetto a $s_1$, cioè tutti i valori $\geq s_1$, viene partizionato in due sottoregioni (due sotto figli) secondo il predicato "$x < s_3$" tali che
1. la prima conterrà i valori compresi fra $s_1$ e $s_3$
2. la seconda conterrà i valori maggiori di $s_3$.
<img src="imgs/splitting3.png" alt="Ulteriore suddivisione della regione di destra" width=400>

Al termine di questa procedura abbiamo ottenuto quatto foglie (o sottospazi del dataset), ciascuno con una propria retta di regressione che approssima entro un errore prestabilito, tale regione. 

#### Due feature
Se avessimo un dominio a due feature $x_1, x_2$ ci troveremmo in uno spazio a tre dimensioni, due per le feature e una per la variabile $y$ target.
<img src="imgs/alberiduefeature.png" alt="Rappresentazione grafica modello a due feature" width=150>
Escludendo la variabile target, il grafico dello spazio sarebbe il seguente 
<img src="imgs/alberiduefeaturebidim.png" alt="Rappresentazione grafica modello a due feature" width=200>
nel quale una prima partizione viene fatta su una delle due variabili, ad esempio $x_1 =$ età, splittando la regione in quei valori tali per cui $x_1 < t_1$ (dove per continuare l'esempio la soglia $t_1 = 15$ anni), che genera una retta verticale che divide lo spazio in due regioni e il secondo split viene fatto sulla seconda variabile $x_2 < t_2$ (con soglia $t_2 =$ maschio). Questo secondo split divide la regione sinistra del primo split, nuovamente in due regioni $R1, R2$. A questo punto la regione di destra rispetto alla prima soglia $t_1$ viene nuovamente suddivisa rispetto alla variabile $x_1$ su una soglia $t_3$ per cui avremo una regione in cui $t_1 \leq x < t_3$ e una in  cui $x \geq t_3$. La regione a destra di questo nuovo split viene ulteriormente divisa, questa volta rispetto alla variabile $x_2$ in una soglia $t_4$. 
Il risultato, dal punto di vista dell'albero binario sarà il seguente
<img src="imgs/alberiduefeaturebinario.png" alt="Rappresentazione binaria modello a due feature" width=250>
Poiché ci troviamo in uno spazio tridimensionale, in realtà il significato geometrico della regressione non è più una retta bensì un piano ortogonale a $y$.
Ad esempio nella seguente immagine
<img src="imgs/treeconsumi.png" alt="Rappresentazione grafica modello consumi" width=550>
non facciamo altro che stimare il consumo in miglia per gallone MPG ($y$) sulla base di due feature (sugli assi orizzontali) che sono potenza in cavalli e peso del veicolo ($X$). Vediamo che ogni istanza è un punto e l'albero di regressione è dato dall'insieme di tutti i piani di regressione (ortogonali rispetto alla variabile MPG da predire). In questo modo, affettando lo spazio con tante regressioni, si ottiene l'errore più piccolo possibile nel prevedere il consumo per ciascuno dei punti forniti in input.

#### Algoritmo greedy di base
L'algoritmo è definito come segue
- Sia $k = 0$ una variabile di iterazione (serve per tenere il conto degli split che facciamo e $S = \mathbb{R}^d$ lo spazio iniziale dei dati, dove $d =$ numero di variabili in input. Ovviamente, dunque, $d$ è il numero di assi dello spazio vettoriale in questione.
- L'algoritmo sarà il seguente 

def ***RegressionTree***$(k, S)$:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$k += 1$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;per ogni variabile di input $j = 1, \dots, d$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;divide lo spazio $S$ scegliendo per $j$ il valore di separazione $v_j \in \mathbb{R}$ t.c. 
$$
    S_{j, <} = \{i | x_{i,j} < v_j \}
$$

$$
    S_{j, \geq} = \{i | x_{i,j} \geq v_j \} 
$$

$$
\overline{y}_{<} = \frac{\sum_{i \in S_{j, <}}(y_i)}{|S_{j, <}|}
$$

$$
\overline{y}_{\geq} = \frac{\sum_{i \in S_{j, \geq}}(y_i)}{|S_{j, \geq}|}
$$

$$
MSE_j = \text{min }\sum_{i \in S_{j, <}} (y_i - \overline{y}_<)^2 + \sum_{i \in S_{j, \geq}} (y_i - \overline{y}_{\geq})^2
$$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;sia $p$ la variabile in $1, \dots, d$ che ha ottenuto il minimo $MSE_p$ 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;se la *condizione di stop* è insoddisfatta

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ***return RegressionTree***$(k, S_{p, <}) \cup$ ***RegressionTree***$(k, S_{p, \geq})$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;altrimenti

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;***return***[($p, v_p$)]

In pratica itero su ogni variabile di input $j$ fra tutte le $d$ possibili. Per ogni $j$ divido lo spazio scegliendo il valore di separazione $v_j$ reale, tale per cui creiamo due sottospazi 
- $S_{j, <}$ di tutte le istanze $i$ che hanno feature $j$ strettamente minore dell'elemento soglia $v_j$
- $S_{p, \geq}$ di tutte le istanze $i$ che hanno feature $j$ maggiore o uguale a tale elemento

Inoltre ci calcoliamo il valore medio della $y$ target di tutte le istanze, il quale sarà
- per il primo insieme $\overline{y}_<$, calcolato come la somma di tutti i valori $y_i$ appartenenti al primo spazio fratto il numero (il modulo conta il numero di elementi) appartenenti a questo insieme.
- per il secondo insieme $\overline{y}_{\geq}$, calcolato analogamente al primo, ma ovviamente con le $y$ del secondo spazio.

A questo punto applichiamo i minimi quadrati ($MSE_j$), andando a minimizzare la funzione d'errore data da due regressioni in questo caso, cioè quella di sinistra, appartenente allo spazio $S_{j, <}$ e quella di destra di $S_{j, \geq}$

$$
\sum_{i \in S_{j, <}} (y_i - \overline{y}_<)^2 + \sum_{i \in S_{j, \geq}} (y_i - \overline{y}_{\geq})^2
$$

A questo punto prendo la variabile di input $p$ che ha ottenuto l'errore quadratico minimo. Questo ci fa capire anche come sono scelte le feature su cui splittare, se ad esempio il primo elemento che ha dato il minimo $MSE_j$ è della feature temperatura, io splitterò prima su quella che sul giorno della settimana e viceversa.
Ovviamente si splitta sulla variabile (o feature) $p = v_j$, cioè la feature $p$ che assume valore $v_j$. 

A questo punto mi chiedo se la *condizione di stop* (che può variare da algoritmo ad algoritmo) è *insoddisfatta*, significa che dovrò splittare ancora sui due nodi figli di quello in cui mi trovo ora, lo faccio ricorsivamente ritornando i valori restituiti dalla chiamata della funzione in questione sui due sottospazi $S_{p, <}, S_{p, \geq}$ generati dallo split su $v_j$ dell'$S_j$ attuale. Se invece la condizione è soddisfatta restituisco la feature $p$ che si è scelta in questa iterazione e il valore da essa assunto come *split*, ad esempio potrebbe ritornare [('temperatura', 15)].

La *condizione di stop* non è definita a priori ma dipende dall'algoritmo utilizzato. Si determina tipicamente mediante iperparametri, come ad esempio la massima profondità $k$ dell'albero, cioè si pone un limite al numero di iterazioni, o massimo numero di foglie, o un numero minimo di istanze per foglia o valore minimo di $MSE$.

#### Complessità del modello e Overfitting
Negli algoritmi di ML esiste *sempre* uno o più iperparametri (o "manopole") che consentono di regolare la complessità del modello. Per quanto riguarda gli algoritmi di regressione questi iperparametri tipicamente regolano il grado del polinomio. Nel caso degli alberi di regressione le modalità di regolazione consiste nella *dimensione dell'albero di regressione*, che consiste in 
- profondità (cioè numero di nodi in verticale)
- numero di foglie

In generale tanto più vogliamo complicare il modello, quanto più grande (sia in termini di profondità che numero di foglie) dev'essere l'albero.

Il fatto di poter aumentare la complessità del modello a piacere, è consentito unicamente dalla possibilità di fare *regolarizzazione* dei parametri, altrimenti diventerebbero troppo grandi e si correrebbe il rischio di avere overfitting.

### Come gestire la complessità di un modello
Qualora avessi un modello con tante features, dunque tantissimi assi e pertanto fosse difficile visualizzarlo graficamente, come posso capire se la complessità del modello è giusta per quanto debba fare?
Un'idea può essere quella di aumentare tale complessità fino a portare il modello in *overfitting* rispetto al training set e dunque ridurre l'errore al minimo (prossimo a zero) sul training set e ovviamente avere un errore elevato sul validation set. Questo però mi fa capire che ho trovato la complessità giusta per modellare bene il mio training set e dunque comincio a lavorare sulla ***regolarizzazione***, cioè tengo fisso il modello che ho trovato, e provo a modificare l'iperparametro che regola la regolarizzazione. Provo ad esempio una regressione L1 (cioè norma 1) LASSO o una regressione L2 (norma 2) Ridge e vedo come varia il coefficiente $R^2$. 

**Riassumendo** In ogni algoritmo di ML la prassi è la seguente
1. Provo ad aumentare la complessità del modello fintanto che l'errore sui dati di training non si minimizza (e nel 90% dei casi andrò in overfitting), se con la complessità che ho l'errore ancora non mi soddisfa continuo ad aumentare la complessità.
2. Una volta raggiunta la giusta complessità del modello per modellare correttamente i miei dati di training controllo come si comporta sul dataset di validation, qualora l'erroe sia notevolmente più basso rispetto a quello ottenuto sul training set, sarò in overfitting. Se questo accade occorre regolarizzare.
Ad esempio nella regressione tipicamente la regolarizzazione avveniva sommando alla funzione d'errore da minimizzare anche un $\lambda$ moltiplicato per la norma 2 al quadrato, o norma 1 (o entrambe bilanciate con un ulteriore iperparametro $\alpha$) dei parametri $\theta$.
Ad esempio nella regressione elastic net, che combina ridge e lasso si doveva minimizzare la funzione

$$
\text{min }E(\theta) = \sum_{i=1}^m (\theta^T x^{(i)} - y^{(i)})^2 + \lambda (\alpha \| \theta \|_1 + (1-\alpha) |\theta\|_2^2)
$$

#### Gestire la complessità negli alberi di regressione
Nel caso degli alberi di regressione la regolarizzazione si applica utilizzando un iperparametro ***C***, tale per cui occcorre minimizzare 

$$
    \text{min} E =  \text{errore-di-training}(Tree) + C \cdot \text{dimensione}(Tree) 
$$

In altre parole occorre minimizzare l'errore in fase di training (aumentando la complessità del modello) ma anche il prodotto fra l'iperparametro C e la dimensione dell'albero. Sappiamo che, più la dimensione dell'albero è elevata e meno sarà regolarizzato. Inoltre il parametro C consente di pesare a piacere l'importanza che si da a tale dimensione.
In altre parole occorre trovare quell'albero più piccolo possibile che comunque minimizzi l'errore di training.

Esistono diverse strategie che possono aiutare in questo senso
- Lasciare crescere l'albero da zero e fermarlo quando la *Loss* aumenta. Sappiamo che nel training set la Loss (cioè l'errore commesso dal modello) diminuisce sempre tendendo a zero. Tuttavia arriverà un certo punto in cui l'ho fatto crescere cosi tanto da aver saturato anche il training set stesso aumentando cosi la loss anche su di esso.
- Lasciar crescere completamente l'albero per poi potarlo partendo dalle foglie fino a quando la loss torna ad aumentare. Potare significa prendere due istanze figlie di un nodo e riportarle al padre, cioè annullare lo split. Ovviamente facendo questo, arriverò ad un certo punto a rendere il modello meno preciso facendo si che la loss ritorni ad aumentare (andando in underfitting: modello troppo semplice per i nostri dati). Questa scelta è meno sensibile a scelte greedy non ottimale di predicati (cioè l'algoritmo visto prima,  il quale parte a splittare dal primo attributo che mi fa ridurre la loss). 

#### Regole di potatura
Esistono diverse regole per potare gli alberi
- Stop quando ogni foglia continee una sola istanza o un numero di istanze inferiore a una certa soglia.
- Stop quando il numero di foglie è inferiore ad una soglia o quando l'errore nella foglia è inferiore a una certa soglia. 
- Stop quando il *p-value* (indica quanto la differenza fra due insiemi di dati originati da uno splitting è frutto del caso) nella divisione di un nodo è maggiore di una soglia (ad esempio 0.05) calcolata sulla base di un *test statistico*. Ad esempio uno di questi test è il **Student T test** che, come vedremo dopo, consente di stabilire se due partizioni di dati sono diverse con significatività statistica dovuta al p-value stesso: non siamo più noi a decidere le soglie di splitting, bensì grazie a questo test stabilisco se le due medie ottenute dai due insiemi sono statisticamente significative.
- **Albero di classificazione**: tipicamente gli alberi in ML sono utilizzati a due scopi, o quando occorre prevedere una variabile continua (come nel caso della regressione) e in questo caso si parla di *alberi di regressione*, oppure quando bisogna prevedere l'esito di una variabile in un dominio discreto (cioè classificazione), ad esempio se un utente della banca restituirà o meno un prestito, ecc.), in tal caso vengono definiti *alberi di classificazione*. In questo caso l'obbiettivo è separare al meglio le istanze. Il criterio di arresto potrebbe essere quello di averre istanze nella stessa foglia che possiedono stesso label. Ad esempio in una foglia coloro che non restituiscono il prestito e nell'altra quelli che lo fanno.

#### Student T Test  e potatura di 2 insiemi
Test usato tutte le volte che dobbiamo fare un test per decidere se la differenza fra due set di dati è significativa o frutto del caso. In questo contesto si hanno
- **Ipotesi da testare**. In particolare questa ipotesi è detta *nulla* poiché da testare è se le medie dei due gruppi sono equivalenti. In caso affermativo posso potare le due piante poiché non ha senso suddividere i dati.
- **T-Score**. Devo decidere che criterio applicare per definire la differenza fra due medie.  Questo score, che non è altro che il rapporto fra la differenza fra gruppi di dati e la differenza interna a ciascun gruppo, serve proprio a questo. 
- Maggiore è questo t-score, cioè maggiore è la differenza fra i due gruppi, e minore sarà la probabilità, anche detta **P-Value** che la correlazione statistica fra i due gruppi sia frutto del caso. In altre parole maggiore è t-score e più è probabile che non vi sia correlazione fra elementi appartenenti ciascuno a uno dei due insiemi, pertanto lo split è adeguato.
- Il p-value è calcolato con **T-Test Student**.  
Per fare questo test occorre fissare (come iperparametro) una **soglia di accettazione** dell'ipotesi nulla (ad esempio 0.05) che indica la soglia entro cui p-value può stare prima di affermare che l'ipotesi nulla è accaduta. In altre parole se il corrispondente p-value è superiore a tale soglia, allora assumiamo che la differenza tra i due gruppi sia frutto del caso. Qualora l'ipotesi nulla si verifichi, come già detto, i due gruppi sono equivalenti e non ha senso mantenere lo split: possiamo riunirli nel nodo padre potando le due foglie.

Dal punto di vista matematico il calcolo di **t-score** e **p-value** è ottenuto come segue:

Siano 2 foglie $S_{j, <}, S_{j, \geq}$ date dalla suddivisione rispetto a $v_j \in \mathbb{R}$ tali che, come sempre
- $S_{j, <} = \{ i | x_{i,j} < v_j\}$
- $S_{j, \geq} = \{ i | x_{i,j} \geq v_j\}$

e le medie dei valori target (cioè delle $y$) saranno rispettivamente
- $\overline{y}_{<} = \frac{\sum_{i \in S_{j, <}} (y_i)}{|S_{j,<}|}$, cioè la somma di tutti i valori target le cui variabili $x_{i,j}$ di input siano minori di $v_j$ (dove $x_{i,j}$ indica il valore assunto dall'istanza $i$-esima per la feature $j$-esima)
- $\overline{y}_{\geq} = \frac{\sum_{i \in S_{j, \geq}} (y_i)}{|S_{j,\geq}|}$. Analogo a sopra.

A questo punto calcoliamo anche le varianze come
- $\sigma_{\overline{y}_{<}}^2 = \frac{\sum_{i \in S_{j, <}} (y_i - \overline{y}_<)^2}{|S_{j, <} - 1|}$
- $\sigma_{\overline{y}_{\geq}}^2 = \frac{\sum_{i \in S_{j, \geq}} (y_i - \overline{y}_{\geq})^2}{|S_{j, \geq} - 1|}$


$\overline{y}_{<} - \overline{y}_{\geq}$